# 🧬 Módulo 4: Modelado de Proteínas y Docking Molecular
## Actividad 4.1: Fundamentos de Estructura de Proteínas

<div align="center">
  
**Universidad de Caldas - Departamento de Química**  
*Introducción a la Química Computacional (173G7G)*  
**Profesor:** José Mauricio Rodas Rodríguez

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/maurorodas/Quimica_computacional_173G7G/blob/main/modulo_04_modelado_proteinas_docking/01_fundamentos_estructura_proteinas.ipynb)

</div>

---

## 🎯 Objetivos de Aprendizaje

Al finalizar esta actividad, serás capaz de:
- Comprender los niveles de estructura proteica (primaria, secundaria, terciaria, cuaternaria)
- Descargar estructuras desde PDB y AlphaFold Database
- Visualizar proteínas en 3D con Py3Dmol
- Analizar estructuras con BioPython
- Extraer secuencias y analizar calidad estructural
- Interpretar valores de confianza (pLDDT) en modelos de AlphaFold

---

## 1. Instalación de Dependencias

In [ ]:
# Instalación de paquetes necesarios
!pip install biopython py3Dmol requests numpy pandas matplotlib

In [ ]:
# Importación de bibliotecas
import requests
import py3Dmol
from Bio import PDB
from Bio.PDB import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

print("Bibliotecas importadas correctamente")

## 2. Niveles de Estructura Proteica

Las proteínas tienen cuatro niveles de organización estructural:

1. **Estructura Primaria**: Secuencia de aminoácidos
2. **Estructura Secundaria**: Hélices α, láminas β, loops
3. **Estructura Terciaria**: Plegamiento 3D completo
4. **Estructura Cuaternaria**: Ensamblaje de múltiples cadenas

## 3. Descarga de Estructuras desde PDB

In [ ]:
def descargar_pdb(pdb_id, output_dir="estructuras"):
    """
    Descarga un archivo PDB desde RCSB
    
    Args:
        pdb_id: Código PDB de 4 caracteres
        output_dir: Directorio de salida
    
    Returns:
        Path al archivo descargado
    """
    Path(output_dir).mkdir(exist_ok=True)
    
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    response = requests.get(url)
    
    if response.status_code == 200:
        output_path = Path(output_dir) / f"{pdb_id}.pdb"
        with open(output_path, 'w') as f:
            f.write(response.text)
        print(f"✓ Estructura {pdb_id} descargada correctamente")
        return output_path
    else:
        print(f"✗ Error al descargar {pdb_id}")
        return None

In [ ]:
# Ejemplo: Descargar lisozima (1AKI)
pdb_file = descargar_pdb("1AKI")

## 4. Visualización 3D con Py3Dmol

In [ ]:
def visualizar_proteina(pdb_file, style="cartoon", color="spectrum"):
    """
    Visualiza una proteína usando Py3Dmol
    """
    with open(pdb_file, 'r') as f:
        pdb_data = f.read()
    
    view = py3Dmol.view(width=800, height=600)
    view.addModel(pdb_data, 'pdb')
    
    if style == "cartoon":
        view.setStyle({'cartoon': {'color': color}})
    elif style == "stick":
        view.setStyle({'stick': {}})
    elif style == "sphere":
        view.setStyle({'sphere': {}})
    
    view.zoomTo()
    return view

In [ ]:
# Visualizar la proteína descargada
view = visualizar_proteina(pdb_file, style="cartoon", color="spectrum")
view.show()

## 5. Análisis de Estructura con BioPython

In [ ]:
# Parsear el archivo PDB
parser = PDB.PDBParser(QUIET=True)
structure = parser.get_structure("proteina", pdb_file)

# Información básica
print(f"ID de la estructura: {structure.id}")
print(f"Número de modelos: {len(structure)}")

# Información de cadenas
for model in structure:
    print(f"\nModelo: {model.id}")
    for chain in model:
        num_residuos = len(list(chain.get_residues()))
        print(f"  Cadena {chain.id}: {num_residuos} residuos")

## 6. Extracción de Secuencia

In [ ]:
def extraer_secuencia(structure, chain_id='A'):
    """
    Extrae la secuencia de aminoácidos de una cadena
    """
    from Bio.PDB.Polypeptide import PPBuilder
    
    ppb = PPBuilder()
    for chain in structure[0]:
        if chain.id == chain_id:
            for pp in ppb.build_peptides(chain):
                return pp.get_sequence()
    return None

# Extraer secuencia de la cadena A
secuencia = extraer_secuencia(structure, 'A')
print(f"Secuencia (primeros 50 residuos): {secuencia[:50]}")
print(f"Longitud total: {len(secuencia)} residuos")

## 7. Análisis de Estructura Secundaria

In [ ]:
# Usar DSSP para análisis de estructura secundaria (si está disponible)
# Nota: DSSP debe estar instalado en el sistema

try:
    from Bio.PDB.DSSP import DSSP
    
    model = structure[0]
    dssp = DSSP(model, pdb_file, dssp='mkdssp')
    
    # Contar elementos de estructura secundaria
    helix_count = sum(1 for res in dssp if res[2] == 'H')
    sheet_count = sum(1 for res in dssp if res[2] == 'E')
    coil_count = sum(1 for res in dssp if res[2] == '-')
    
    print(f"Hélices α: {helix_count} residuos")
    print(f"Láminas β: {sheet_count} residuos")
    print(f"Loops/coils: {coil_count} residuos")
    
except:
    print("DSSP no está disponible. Instálalo para análisis de estructura secundaria.")

## 8. Descarga desde AlphaFold Database

In [ ]:
def descargar_alphafold(uniprot_id, output_dir="estructuras"):
    """
    Descarga una estructura predicha por AlphaFold
    
    Args:
        uniprot_id: ID de UniProt
        output_dir: Directorio de salida
    """
    Path(output_dir).mkdir(exist_ok=True)
    
    url = f"https://alphafold.ebi.ac.uk/files/AF-{uniprot_id}-F1-model_v4.pdb"
    response = requests.get(url)
    
    if response.status_code == 200:
        output_path = Path(output_dir) / f"AF-{uniprot_id}.pdb"
        with open(output_path, 'w') as f:
            f.write(response.text)
        print(f"✓ Estructura AlphaFold {uniprot_id} descargada")
        return output_path
    else:
        print(f"✗ Error al descargar {uniprot_id} de AlphaFold")
        return None

In [ ]:
# Ejemplo: Descargar proteína humana
# P53_HUMAN = P04637
af_file = descargar_alphafold("P04637")

## 9. Análisis de Confianza en Modelos AlphaFold

In [ ]:
def analizar_plddt(pdb_file):
    """
    Analiza los valores de pLDDT (confianza) en un modelo de AlphaFold
    """
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure("AF", pdb_file)
    
    plddt_values = []
    residue_numbers = []
    
    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    if atom.name == 'CA':  # Solo carbono alfa
                        plddt_values.append(atom.bfactor)
                        residue_numbers.append(residue.id[1])
                        break
    
    return residue_numbers, plddt_values

In [ ]:
if af_file:
    residues, plddt = analizar_plddt(af_file)
    
    plt.figure(figsize=(12, 4))
    plt.plot(residues, plddt, linewidth=2)
    plt.axhline(y=70, color='r', linestyle='--', label='Confianza alta (>70)')
    plt.axhline(y=50, color='orange', linestyle='--', label='Confianza media (50-70)')
    plt.xlabel('Número de Residuo')
    plt.ylabel('pLDDT Score')
    plt.title('Confianza del Modelo AlphaFold (pLDDT)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print(f"pLDDT promedio: {np.mean(plddt):.2f}")
    print(f"Residuos con alta confianza (>70): {sum(p > 70 for p in plddt)}")
    print(f"Residuos con baja confianza (<50): {sum(p < 50 for p in plddt)}")

## 10. Ejercicios Prácticos

### Ejercicio 1
Descarga la estructura de la proteína spike del SARS-CoV-2 (PDB: 6VXX) y visualízala.

### Ejercicio 2
Compara una estructura experimental del PDB con su correspondiente predicción de AlphaFold.

### Ejercicio 3
Analiza la distribución de elementos de estructura secundaria en una proteína de tu elección.

In [ ]:
# Espacio para tus ejercicios

## Referencias

1. Berman et al. (2000). The Protein Data Bank. *Nucleic Acids Research*
2. Jumper et al. (2021). Highly accurate protein structure prediction with AlphaFold. *Nature*
3. BioPython Documentation: https://biopython.org/
4. RCSB PDB: https://www.rcsb.org/

---

## 📚 Recursos Adicionales

### Bases de Datos
- [Protein Data Bank (PDB)](https://www.rcsb.org/)
- [AlphaFold Database](https://alphafold.ebi.ac.uk/)
- [UniProt](https://www.uniprot.org/)

### Tutoriales
- [BioPython Tutorial](https://biopython.org/wiki/Documentation)
- [PDB File Format](http://www.wwpdb.org/documentation/file-format)
- [Py3Dmol Documentation](https://3dmol.csb.pitt.edu/)

---

## ✅ Verificación de Aprendizaje

Al finalizar esta actividad deberías ser capaz de:

- ✅ Descargar estructuras de proteínas desde PDB y AlphaFold
- ✅ Visualizar y analizar estructuras en 3D
- ✅ Extraer secuencias y analizar estructura secundaria
- ✅ Interpretar valores de confianza (pLDDT)
- ✅ Identificar regiones estructuradas y desordenadas

---

<div align="center">

## 🎉 ¡Felicitaciones!

Has completado la **Actividad 4.1: Fundamentos de Estructura de Proteínas**

[![Siguiente](https://img.shields.io/badge/Actividad_4.2_➡️-Modelado_por_Homología-green.svg)](02_modelado_homologia.ipynb)

---

📚 **[Volver al Módulo 4](README.md)** | 🏠 **[Inicio del Curso](../README.md)**

---

**Universidad de Caldas - Departamento de Química**  
*Química Computacional 173G7G*

</div>